In [ ]:
cd ..

In [ ]:
import os
import numpy as np
from src.analyzer import DataAnalyzer
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
output_dir = 'output/motion_benefit4'

In [ ]:
# output_dir = 'output/sparsity'

In [ ]:
pkl_fns = [os.path.join(output_dir, fn) 
           for fn in os.listdir(output_dir) 
           if fn.endswith('.pkl')]
pkl_fns.sort()
len(pkl_fns)

In [ ]:
da_ = [DataAnalyzer.fromfilename(pkl_fn) for pkl_fn in pkl_fns[21:22]]

In [ ]:
da = da_[0]

In [ ]:
# fp = da.data['FP']

# plt.hist(fp.ravel(), bins=50)
# plt.yscale('log')

In [ ]:
# ax = plt.axes()
# ips = da.data['Ips'][0, :, 50]
# xe = da.data['XE']
# ye = da.data['YE']
# ie = da.data['IE']
# de = da.data['de']

# m = abs(ips).max()

# for x, y, i, val in zip(xe, ye, ie, ips):
#     ax.add_patch(plt.Circle((x, -y), de * 0.203, alpha=val/m))

# rng = max(max(xe), max(ye))
# ax.set_xlim([-rng, rng])
# ax.set_ylim([-rng, rng])
# ax.set_aspect('equal')

In [ ]:
plt.rcParams['font.size'] = 7

In [ ]:
0 if True else 1

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, sharey=True,
                         figsize=(3.5, 1))

for i, q in enumerate([10, 50, 100]):
    da.plot_image_estimate(fig, axes[i], q=q,
                           colorbar=(i==2))
# plt.tight_layout()
plt.savefig('output/improvement.pdf')

In [ ]:
fig, axes = da.plot_em_estimate(100, figsize=(3.5, 5))

axes[2][1].set_ylabel('')


axes[0][0].set_ylabel('Position (arcmin)')
axes[1][0].set_ylabel('Position (arcmin)')

start, stop = axes[0][1].get_xlim()
ticks = np.arange(start, stop + 0.5, 2)
axes[0][1].set_xticks(ticks)
axes[0][1].set_yticks(ticks)

plt.tight_layout()
plt.savefig('output/example.pdf')
# for _ in axes.flat:
#     pass

In [ ]:
fig, ax = plt.subplots(1, 2)
# ax.fill_between(np.arange(4), np.zeros(4), np.ones(4))
ax[0].set_xlabel('hi')
cax = ax[0].imshow(np.arange(9).reshape(3, 3))
fig.colorbar(cax, ax=ax[0])

In [ ]:
fig, ax = plt.subplots(figsize=(0.75, 0.75))
ax.set_xlabel(None, fontsize=7)
ax.set_ylabel(None, fontsize=7)

ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')



# ax.get_xaxis().set_ticks([])
start, stop = ax.get_xlim()
xres = 0.5
ticks = np.arange(start, stop + xres, xres)
ax.set_xticks(ticks)

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(7)
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(7)

    
da.plot_tuning_curves()

start, stop = ax.get_ylim()
xres = 40
ticks = [10, 50, 100]
ax.set_yticks(ticks)

ax.set_ylabel('')
ax.set_xlabel('')

ax.legend().set_visible(False)
ax.title.set_fontsize(7)
plt.tight_layout(pad=0.3)
plt.savefig('output/firing_rates.pdf')

In [ ]:
start

In [ ]:
ax.get_ylim()

In [ ]:
save_dir = 'output/spike_video/'

In [ ]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
da.plot_image_and_rfs(alpha_rf=0.25)

In [ ]:
for i, q in enumerate(range(0, da.N_T, 1)):
# for i, q in enumerate(range(0, 1, 1)):
    print 'Creating figure {:04d}'.format(q)
    da.plot_moving_image_and_spikes(q)
    plt.suptitle('Image Projected on the Retina and Generated Spikes at t = {:03d} ms'.format(q))
    plt.savefig(os.path.join(save_dir, 'img_{:04d}'.format(i)))
    plt.close()

To synthesize the different png files run:

`
avconv -framerate 20 -i img_%04d.png -c:v libx264 -r 30 video/out.mp4
`